# Insights Assistant
This is a small tool built with LangChain that helps you summarize quarterly reports based on monthly reports.
## How to use
* Each time you send your monthly report email, also forward it to the Insights Assistant inbox.
* When a quarterly report is needed, run the Insights Assistant.
* The quarterly report will be automatically sent to your work email.

Settings of OpenAI key and Email account

In [ ]:

import os

# setting OpenAI Key and Base URL（if you are using Qwen）
os.environ["OPENAI_API_KEY"] = "sk..."  # ✅ Replace with yourss
os.environ["OPENAI_BASE_URL"] = "https://dashscope.aliyuncs.com/compatible-mode/v1"

# # Gmail IMAP
# IMAP_SERVER = "imap.gmail.com"
# ASSITANT_EMAIL_ACCOUNT = "xxx@gmail.com"  # ✅ Replace with your email.
# ASSITANT_EMAIL_PASSWORD = "xxx"  # ✅ Replace with your password
# SUBJECT_KEYWORD = "月报"  #  ✅ Replace with your monthly report keywork
# WORK_EMAIL_ACCOUNT = "xxx@gmail.com"  # ✅ Replace with your email
# SMTP_SSL_SERVER = "smtp.gmail.com"
# SMTP_SSL_PORT = 465


Addding tool of get monthly report

In [2]:
import re
from datetime import datetime
import email
from email.header import decode_header
from imapclient import IMAPClient
from langchain.tools import tool

@tool
def fetch_monthly_reports(input: str):
    """连接 Gmail 并获取主题包含月报相关内容的邮件内容, input 字符串为占位用"""
    with IMAPClient(IMAP_SERVER) as client:
        client.login(ASSITANT_EMAIL_ACCOUNT, ASSITANT_EMAIL_PASSWORD)
        client.select_folder("inbox")
        client.capabilities()

        search_criteria = ["SUBJECT", SUBJECT_KEYWORD, "SINCE", "15-Jan-2025"]

        email_ids = client.search(search_criteria, charset="utf-8")
        if not email_ids:
            print("❌ 邮件搜索失败。")
            return []

        monthly_data = []


        for e_id in email_ids:
            msg_data = client.fetch([e_id], ["RFC822"])
            for msg_id, msg in msg_data.items():
                msg = email.message_from_bytes(msg[b"RFC822"])

                
                date_tuple = email.utils.parsedate_tz(msg["Date"])
                email_date = (
                    datetime.fromtimestamp(email.utils.mktime_tz(date_tuple))
                    if date_tuple
                    else None
                )

                
                subject, encoding = decode_header(msg["Subject"])[0]
                if isinstance(subject, bytes):
                    subject = subject.decode(encoding or "utf-8")

                def clean_text(text):
                    return re.sub(r"\s+", " ", text).strip()

                subject = clean_text(subject)

                content = ""
                if msg.is_multipart():
                    for part in msg.walk():
                        if (
                            part.get_content_type() == "text/plain"
                            and "attachment" not in str(part.get("Content-Disposition"))
                        ):
                            charset = part.get_content_charset()
                            content = part.get_payload(decode=True).decode(
                                charset or "utf-8", errors="ignore"
                            )
                            break
                else:
                    charset = msg.get_content_charset()
                    content = msg.get_payload(decode=True).decode(
                        charset or "utf-8", errors="ignore"
                    )
                content = clean_text(content)

                monthly_data.append(
                    {"date": email_date, "subject": subject, "content": content}
                )

    return monthly_data


Addding tool of sending quarterly report

In [3]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import json


@tool
def send_summary_email(input: str) -> str:
    """发送季度总结到工作邮箱 JSON 字符串，包含 subject, content, to_email字段"""
    data = json.loads(input)
    subject = data["subject"]
    content = data["content"]
    to_email = data["to_email"]
    msg = MIMEMultipart()
    msg["From"] = ASSITANT_EMAIL_ACCOUNT
    msg["To"] = to_email
    msg["Subject"] = subject

    msg.attach(MIMEText(content, "plain", "utf-8"))

    try:
        with smtplib.SMTP_SSL(SMTP_SSL_SERVER, SMTP_SSL_PORT) as server:
            server.login(ASSITANT_EMAIL_ACCOUNT, ASSITANT_EMAIL_PASSWORD)
            server.send_message(msg)
        return f"✅ 邮件已成功发送到 {to_email}"
    except Exception as e:
        return f"❌ 邮件发送失败: {str(e)}"


Create Agent

In [4]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

def generate_quarterly_report():
    """调用 LLM 生成季度总结"""
    prompt_template = """
你每个月要写月报，每个季度要写季度总结，季度总结是对每个月月报的总结和提炼。
你需要根据月报内容，智能识别它们属于哪个季度，然后撰写一份最近的季度的总结。
月报可以通过工具去获取，季度总结需要你自己撰写。
总结应包括：每月重点工作、整体成果、存在的问题、改进建议。
总结完之后，请主动将它通过邮件发给 {email}，标题为“自动生成的季度总结”。

调用发送邮件工具时，请传入以下 JSON 格式字符串作为参数：
{{
  "subject": "某年某季度总结",
  "content": "你生成的内容",
  "to_email": "{email}"
}}
    """.strip()

    final_prompt = prompt_template.format(email=WORK_EMAIL_ACCOUNT)

    tools = [fetch_monthly_reports, send_summary_email]

    llm = ChatOpenAI(model_name="qwen-max")
    agent = initialize_agent(
        tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )

    return agent.invoke(final_prompt)


In [5]:
def main():
    generate_quarterly_report()


if __name__ == "__main__":
    main()

/tmp/ipykernel_1131764/390832497.py:26: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
我需要首先通过调用 fetch_monthly_reports 来获取最近几个月的月报内容，然后基于这些信息来分析它们属于哪个季度，并据此撰写季度总结。最后，我会使用 send_summary_email 功能将这份总结发送给指定邮箱。
Action: fetch_monthly_reports
Action Input: "最近三个月"
Observation: [{'date': datetime.datetime(2025, 4, 15, 12, 51, 55), 'subject': '2025年1月月报', 'content': '🔹 *一、本月回顾* 1. 完成「Orion智能加速器」Q1目标拆解与关键里程碑规划； 2. 深度参与架构设计与技术方案评审，明确模块边界与接口； 3. 搭建基础框架，完成初版原型的搭建与验证。 🔹 *二、成果总结* - 输出设计文档2份，覆盖整体架构与关键模块设计； - 初步完成原型开发，测试通过率达90%以上； - 协调跨部门资源，为2月的联调工作打下基础。 🔹 *三、问题与反思* - 任务拆分偏粗，导致部分接口返工； - 初期开发节奏偏快，代码规范需进一步统一。 🔹 *四、下月计划* - 推进主功能模块的开发与单元测试； - 启动首次联调演练，及时处理接口问题； - 完善文档与研发流程，提升团队协作效率'}, {'date': datetime.datetime(2025, 4, 15, 12, 52, 36), 'subject': '2025年2月月报', 'content': '*一、本月回顾* 1. 持续开发并完成「Orion智能加速器」核心模块联调； 2. 优化模型编译逻辑，提升整体执行效率； 3. 引导新人熟悉项目，逐步融入开发流程。 🔹 *二、成果总结* - 模块执行性能提升28%，达成预期优化目标； - 修复关键Bug 5项，提升系统稳定性； - 新成员完成首轮代码提交，成功集成主干。 🔹 *三、问题与反思* - 联调期间暴露部分边缘场景未覆盖，需补充测试用例； - 项目文档更新不及时，影响知识共享效率。 🔹 *四、下月计划* - 推进系统级测试与预发布演练； - 配合产品完成用户演示版本； - 梳理性能瓶颈，为4月迭代做准备。

Email content：
### 一、每月重点工作回顾
- **1月：**
  - 完成「Orion智能加速器」Q1目标拆解与关键里程碑规划。
  - 深度参与架构设计和技术方案评审。
  - 搭建基础框架并完成初版原型验证。
- **2月：**
  - 核心模块联调及优化模型编译逻辑以提升性能。
  - 引导新成员熟悉项目流程。
- **3月：**
  - 第一阶段版本发布。
  - 协助产品团队进行内测演示和支持用户答疑。
  - 启动新版本需求评审。

### 二、整体成果
- 成功完成了从概念到初步实现再到稳定上线的全过程。
- 系统性能显著提升，并达到了预期的所有关键指标。
- 团队内部协作得到了加强，同时通过技术分享增强了对项目的理解。
- 为后续迭代打下了坚实的基础，包括但不限于文档输出、性能优化等。

### 三、存在问题
- 在开发过程中存在任务划分不够精细的问题，导致了部分工作的返工。
- 初始阶段代码规范性不足，需要进一步统一标准。
- 测试用例覆盖不全，在联调期间暴露了一些边缘场景下的问题。
- 文档更新滞后影响了信息共享效率。
- 上线前突发Bug处理给团队带来了较大压力。

### 四、改进建议
- 未来工作中应更加注重前期规划的细致程度，确保每个环节都有明确的目标和责任人。
- 加强代码质量管理，建立更严格的审查机制。
- 增加测试覆盖率，特别是针对复杂或异常情况下的测试。
- 定期更新项目文档，保持其时效性和准确性。
- 建立更有效的预演机制来应对潜在的技术挑战。